In [1]:
!pip install -U pyspark
from pyspark.sql import SparkSession

In [2]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Initiate and configure Spark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

print("Using Apache Spark Version", spark.version)

23/04/28 10:19:42 WARN Utils: Your hostname, MacBook-Pro-540.local resolves to a loopback address: 127.0.0.1; using 10.206.94.115 instead (on interface en0)
23/04/28 10:19:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/28 10:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Apache Spark Version 3.4.0


In [4]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
filecontents = pd.read_csv('cafes1.csv')

schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("Longitude", DoubleType(), True), 
    StructField("Types", StringType(), True), 
    StructField("Price_level", IntegerType(), True), 
    StructField("Rating", IntegerType(), True), 
    StructField("Reviews", IntegerType(), True), 
    StructField("Neighborhood", StringType(), True),  
])

sparkDF=spark.createDataFrame(filecontents, schema) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Types: string (nullable = true)
 |-- Price_level: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Neighborhood: string (nullable = true)



+--------------------+--------------------+-----------------+------------------+--------------------+-----------+------+-------+-------------+
|                Name|             Address|         Latitude|         Longitude|               Types|Price_level|Rating|Reviews| Neighborhood|
+--------------------+--------------------+-----------------+------------------+--------------------+-----------+------+-------+-------------+
|           Cafe Lalo|201 West 83rd Str...|       40.7859528|       -73.9766611|['cafe', 'food', ...|          2|     4|   1197|UpperWestSide|
|      Barnes & Noble|2289 Broadway, Ne...|       40.7862392|       -73.9787281|['book_store', 'c...|          1|     5|   2698|UpperWestSide|
|    Peacefood Uptown|460 Amsterdam Ave...|        40.785275|        -73.977012|['cafe', 'bakery'...|          2|     4|   1384|UpperWestSide|
|        French Roast|2340 Broadway, Ne...|40.78764169999999|-73.97659019999999|['cafe', 'store',...|          2|     4|   1021|UpperWestSide|

In [6]:
sparkDF.unpersist()
sparkDF_tok.unpersist()
sparkDF_tok_wv.unpersist()

NameError: name 'sparkDF_tok' is not defined

In [5]:
sparkDF.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Types: string (nullable = true)
 |-- Price_level: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Neighborhood: string (nullable = true)



In [6]:
from pyspark.sql.functions import col, lower, regexp_replace
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType,StructType, StringType, IntegerType


In [7]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec 
# types
regexTokFilter = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'Types', outputCol = 'tokens_types')
sparkDF_tok= regexTokFilter.transform(sparkDF)

In [8]:
sparkDF_tok.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Types: string (nullable = true)
 |-- Price_level: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- tokens_types: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [9]:
sparkDF_tok['Types','tokens_types'].show()

+--------------------+--------------------+
|               Types|        tokens_types|
+--------------------+--------------------+
|['cafe', 'food', ...|[cafe, food, poin...|
|['book_store', 'c...|[book_store, cafe...|
|['cafe', 'bakery'...|[cafe, bakery, st...|
|['cafe', 'store',...|[cafe, store, res...|
|['cafe', 'store',...|[cafe, store, res...|
|['cafe', 'store',...|[cafe, store, foo...|
|['cafe', 'store',...|[cafe, store, res...|
|['cafe', 'store',...|[cafe, store, foo...|
|['florist', 'cafe...|[florist, cafe, s...|
|['cafe', 'restaur...|[cafe, restaurant...|
|['meal_takeaway',...|[meal_takeaway, c...|
|['cafe', 'restaur...|[cafe, restaurant...|
|['cafe', 'food', ...|[cafe, food, poin...|
|['cafe', 'meal_ta...|[cafe, meal_takea...|
|['cafe', 'bakery'...|[cafe, bakery, st...|
|['restaurant', 'c...|[restaurant, cafe...|
|['cafe', 'store',...|[cafe, store, foo...|
|['bakery', 'cafe'...|[bakery, cafe, st...|
|['cafe', 'meal_ta...|[cafe, meal_takea...|
|['cafe', 'food', ...|[cafe, foo

In [10]:
word2vec_type = Word2Vec(vectorSize = 500, minCount = 1, inputCol = 'tokens_types', outputCol = 'wordvectors_type')
model_type = word2vec_type.fit(sparkDF_tok)
wordvectors_type = model_type.transform(sparkDF_tok)

23/04/28 10:20:15 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [11]:
sparkDF_wv = wordvectors_type.select('Name','Address','Latitude','Longitude','Types','Price_level','Rating',
                                         'Reviews','Neighborhood','tokens_types','wordvectors_type').rdd.toDF()
sparkDF_wv.show()

+--------------------+--------------------+-----------------+------------------+--------------------+-----------+------+-------+-------------+--------------------+--------------------+
|                Name|             Address|         Latitude|         Longitude|               Types|Price_level|Rating|Reviews| Neighborhood|        tokens_types|    wordvectors_type|
+--------------------+--------------------+-----------------+------------------+--------------------+-----------+------+-------+-------------+--------------------+--------------------+
|           Cafe Lalo|201 West 83rd Str...|       40.7859528|       -73.9766611|['cafe', 'food', ...|          2|     4|   1197|UpperWestSide|[cafe, food, poin...|[0.00768792517192...|
|      Barnes & Noble|2289 Broadway, Ne...|       40.7862392|       -73.9787281|['book_store', 'c...|          1|     5|   2698|UpperWestSide|[book_store, cafe...|[0.00607319891181...|
|    Peacefood Uptown|460 Amsterdam Ave...|        40.785275|        -73.97

In [12]:
sparkDF_wv['Types','tokens_types','wordvectors_type'].show()

+--------------------+--------------------+--------------------+
|               Types|        tokens_types|    wordvectors_type|
+--------------------+--------------------+--------------------+
|['cafe', 'food', ...|[cafe, food, poin...|[0.00768792517192...|
|['book_store', 'c...|[book_store, cafe...|[0.00607319891181...|
|['cafe', 'bakery'...|[cafe, bakery, st...|[0.00715056310374...|
|['cafe', 'store',...|[cafe, store, res...|[0.00778069365575...|
|['cafe', 'store',...|[cafe, store, res...|[0.00778069365575...|
|['cafe', 'store',...|[cafe, store, foo...|[0.00722393303876...|
|['cafe', 'store',...|[cafe, store, res...|[0.00778069365575...|
|['cafe', 'store',...|[cafe, store, foo...|[0.00722393303876...|
|['florist', 'cafe...|[florist, cafe, s...|[0.00588488105374...|
|['cafe', 'restaur...|[cafe, restaurant...|[0.00826323948567...|
|['meal_takeaway',...|[meal_takeaway, c...|[0.00655931399523...|
|['cafe', 'restaur...|[cafe, restaurant...|[0.00826323948567...|
|['cafe', 'food', ...|[ca

In [13]:
sparkDF_wv_final = sparkDF_wv.collect()
print(sparkDF_wv_final)

[Row(Name='Cafe Lalo', Address='201 West 83rd Street, New York', Latitude=40.7859528, Longitude=-73.9766611, Types="['cafe', 'food', 'point_of_interest', 'establishment']", Price_level=2, Rating=4, Reviews=1197, Neighborhood='UpperWestSide', tokens_types=['cafe', 'food', 'point_of_interest', 'establishment'], wordvectors_type=DenseVector([0.0077, 0.0061, 0.007, 0.0079, -0.0017, -0.0073, 0.0075, 0.014, -0.0057, -0.005, -0.006, -0.0076, 0.0021, 0.003, -0.0086, 0.0071, 0.0005, -0.0131, 0.0014, 0.0019, -0.0078, 0.0106, -0.0081, -0.0037, -0.0099, 0.0102, -0.0083, -0.0053, 0.0089, -0.0054, -0.0161, -0.0046, 0.0045, 0.012, -0.0098, 0.0108, -0.0126, -0.0048, -0.009, 0.0077, -0.0088, -0.0013, 0.0058, 0.0004, -0.011, -0.0094, 0.0035, -0.0094, 0.0226, 0.0023, 0.0042, -0.0133, 0.0123, 0.0056, -0.0063, 0.0173, -0.0049, -0.0093, 0.0016, 0.0038, -0.0002, 0.0071, -0.0052, -0.0068, 0.0068, -0.0105, 0.0031, -0.0098, -0.0026, -0.0049, -0.0105, 0.0049, -0.0091, 0.0111, 0.0059, -0.0002, 0.0051, -0.0151, -0

In [14]:
synonyms = model_type.findSynonyms("grocery_or_supermarket",10)  
synonyms.show()

+-----------------+--------------------+
|             word|          similarity|
+-----------------+--------------------+
|             cafe|   0.630638062953949|
|convenience_store|   0.585273265838623|
|             food| 0.35125166177749634|
|    establishment|  0.3304271101951599|
|    meal_delivery|  0.1865980327129364|
|      supermarket| 0.07417041808366776|
|     liquor_store|2.582992019597441E-4|
|          florist|-0.05382693558931351|
|           health|-0.09597200900316238|
|      art_gallery|-0.21133248507976532|
+-----------------+--------------------+



In [ ]:
# THE ONE THAT WORKS
from flask import Flask, request, jsonify, redirect, url_for, render_template
import numpy as np

app = Flask("JSON_OUTPUT")

@app.route('/')
def form():
    return render_template('moreedits.html')
        
@app.route('/submit', methods=['GET','POST'])
def submit():
    if request.method == 'POST':
        types_user = request.form['q']
        price_level_user = request.form['price_level']
        neighborhood_user = request.form['neighborhoods']
        rating_user = request.form['rating']
        
        def cossim(v1, v2):
            dot_product = np.sum(v1 * v2)
            mag_v1 = np.sqrt(np.sum(np.power(v1, 2)))
            mag_v2 = np.sqrt(np.sum(np.power(v2, 2)))
            return dot_product / (mag_v1 * mag_v2 + 0.1)

        query_txt = types_user
        query_df = sc.parallelize([(1,query_txt)]).toDF(['index','Types'])
        query_tok = regexTokFilter.transform(query_df)
        query_vec = model_type.transform(query_tok)
        query_vec = query_vec.select('wordvectors_type').collect()[0][0]
        
        sim_rdd = sc.parallelize((i[0],i[1], i[2],i[3],i[4],i[5],i[6],float(cossim(query_vec, i[7])), i[8]) for i in sparkDF_wv_final)
        sim_df = spark.createDataFrame(sim_rdd).\
            withColumnRenamed('_1', 'Name').\
            withColumnRenamed('_2', 'Address').\
            withColumnRenamed('_3', 'Latitude').\
            withColumnRenamed('_4', 'Longitude').\
            withColumnRenamed('_5', 'Types').\
            withColumnRenamed('_6', 'Price_level').\
            withColumnRenamed('_7', 'Rating').\
            withColumnRenamed('_8', 'Similarity').\
            withColumnRenamed('_9', 'Neighborhood').\
            orderBy("Similarity", ascending=False)

    
    pandas_df = sim_df.toPandas()
    df_filtered = pandas_df[pandas_df['Price_level'] == int(price_level_user)]
    df_filtered1 = df_filtered[df_filtered['Rating'] == int(rating_user)]
    df_filtered2 = df_filtered1[df_filtered1['Neighborhood'] == neighborhood_user]
    html_table = df_filtered2.head(10).to_html(classes='table')
    return render_template('onlytable.html', table=html_table)

@app.route('/output')
def output():
    # render the output HTML page
    return render_template('onlytable.html',table=processed_data)
    #return render_template('onlytable.html',table =(sim_df_filtered.to_html(classes='table')))

app.run(host='localhost', port=7030)

 * Serving Flask app "JSON_OUTPUT" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:7030/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Apr/2023 10:20:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:20:49] "GET /static/withblurb.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:20:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2023 10:20:56] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:21:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:21:26] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:21:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:22:07] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:22:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:23:14] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:23:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:23:34] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:24:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 10:42:08] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 

In [30]:
from flask import Flask, request, jsonify, redirect, url_for, render_template
import numpy as np

app = Flask("JSON_OUTPUT")

@app.route('/', methods = ['GET','POST'])
def form():
    if request.method == ['POST']:
        types_user = request.args.get['q']
        #price_level_user = request.args.get['price_level']
        #neighborhood_user = request.args.get['neighborhoods']
        #rating_user = request.args.get['rating']
        
        def cossim(v1, v2): 
            return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))+.1)
        
            query_txt = types_user
            query_df  = sc.parallelize([(1,query_txt)]).toDF(['index','types'])
            query_df = spark.createDataFrame(query_df.rdd, samplingRatio = 0.1)
            query_tok = regexTokFilter.transform(query_df)
            query_swr = stopwordFilter.transform(query_tok)
            query_vec = model_type.transform(query_swr)
            query_vec = query_vec.select('wordvectors_type').collect()[0][0]

        sim_rdd = sc.parallelize((i[0], i[1], i[2], float(cossim(query_vec, i[3]))) for i in sparkDF_wv)
        sim_df  = spark.createDataFrame(sim_rdd).\
                    withColumnRenamed('_1', 'name').\
                    withColumnRenamed('_2', 'address').\
                    withColumnRenamed('_3', 'neighborhoods').\
                    withColumnRenamed('_4', 'price_level').\
                    withColumnRenamed('_5', 'rating').\
                    withColumnRenamed('_6', 'reviews').\
                    withColumnRenamed('_7', 'types').\
                    orderBy("similarity", ascending = False)
        sim_df_filtered = sim_df
        #((sim_df.price_level == price_level_user))
        return redirect(url_for('output', processed_data= sim_df_filtered))   
        
    return render_template('moreedits.html')
    

@app.route('/output')
def output():
    processed_data = request.args.get('sim_df_filtered')
    # render the output HTML page
    return render_template('onlytable.html', table =processed_data)

app.run(host='localhost', port=6057)

 * Serving Flask app "JSON_OUTPUT" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:6056/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2023 20:02:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2023 20:02:00] "GET /withblurb.png HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2023 20:02:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2023 20:02:06] "POST /output HTTP/1.1" 405 -


In [ ]:
# use them in a Flask app to build a user interface for searching cafes based on user input filters
#from flask import Flask, request, jsonify, render_template
#import numpy as np

#app = Flask("JSON_OUTPUT")

#@app.route('/form')
#def form():
    #return render_template('anothertest.html')

#@app.route('/data/', methods=['POST'])
#def data():
    #if request.method == "POST":
        #types_user = request.form.get('q')
        #price_level_user = request.form.get('price_level')
        #neighborhood_user = request.form.get('neighborhood')
        #rating_user = request.form.get('rating')
        
        #def cossim(v1, v2): 
            #return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))+.1)
            #query_txt = types_user
            #query_df  = sc.parallelize([(1,query_txt)]).toDF(['index','input_type'])
            #query_tok = regexTokFilter.transform(query_df)
            #query_swr = stopwordFilter.transform(query_tok)
            #query_vec = model_type.transform(query_swr)
            #query_vec = query_vec.select('wordvectors_type').collect()[0][0]

            #sim_rdd = sc.parallelize((i[0], i[1], i[2], float(cossim(query_vec, i[3]))) for i in sparkDF_wv)
            #sim_df  = spark.createDataFrame(sim_rdd).\
                   #withColumnRenamed('_1', 'name').\
                   #withColumnRenamed('_2', 'address').\
                   #withColumnRenamed('_3', 'neighborhood').\
                   #withColumnRenamed('_4', 'price_level').\
                   #withColumnRenamed('_5', 'rating').\
                   #withColumnRenamed('_6', 'reviews').\
                   #withColumnRenamed('_7', 'types').\
                   #orderBy("similarity", ascending = False)
           # sim_df_filtered = sim_df.filter((sim_df.price_level == price_level_user)|(sim_df.neighborhood == neighborhood_user) &
                          #(sim_df.rating == rating_user))                                                 
        #return render_template('onlytable.html',table =(sim_df_filtered.to_html(classes='table')))
#app.run(host='localhost', port=5013)

In [28]:
from flask import Flask, request, jsonify, redirect, url_for, render_template, json
import numpy as np
import pandas as pd

app = Flask("JSON_OUTPUT")

@app.route('/', methods=['GET','POST'])
def form():
    if request.method == 'POST':
        types_user = request.form['q']
        price_level_user = request.form['price_level']
        neighborhood_user = request.form['neighborhoods']
        rating_user = request.form['rating']
        
        print(types_user)
        print(price_level_user)
        
        # process the data and store it in a variable called processed_data
        processed_data = "This is the processed data."
        
        # redirect to the output route with the processed data as a parameter
        return redirect(url_for('output', processed_data=processed_data))
    
    # if the request method is GET, render the form template
    return render_template('moreedits.html')

@app.route('/output')
def output():
    processed_data = request.args.get('processed_data')
    #print(processed_data)
    # render the output HTML page
    return render_template('onlytable.html', table = processed_data)

app.run(host='localhost', port=6054)

 * Serving Flask app "JSON_OUTPUT" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:6054/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2023 19:58:49] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [27/Apr/2023 19:58:50] "GET /favicon.ico HTTP/1.1" 404 -


In [25]:
from flask import Flask, request, jsonify, redirect, url_for, render_template, json
import numpy as np
import pandas as pd

app = Flask("JSON_OUTPUT")

@app.route('/', methods=['GET','POST'])
def form():
    if request.method == 'POST':
        types_user = request.form['q']
        price_level_user = request.form['price_level']
        neighborhood_user = request.form['neighborhoods']
        rating_user = request.form['rating']
        
        print(types_user)
        print(price_level_user)
        
#         def cossim(v1, v2): 
#             return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))+.1)
        
#         query_txt = bakery
#         query_df = sc.parallelize([(1,query_txt)]).toDF(['index','Types'])
#         query_df = spark.createDataFrame(query_df.rdd, samplingRatio=0.1)
#         query_tok = regexTokFilter.transform(query_df)
#         query_swr = stopwordFilter.transform(query_tok)
#         query_vec = model_type.transform(query_swr)
#         query_vec = query_vec.select('wordvectors_type').collect()[0][0]

#         sim_rdd = sc.parallelize((i[0], i[1], i[2], float(cossim(query_vec, i[3]))) for i in sparkDF_wv)
#         sim_df = spark.createDataFrame(sim_rdd).\
#             withColumnRenamed('_1', 'name').\
#             withColumnRenamed('_2', 'address').\
#             withColumnRenamed('_3', 'neighborhoods').\
#             withColumnRenamed('_4', 'price_level').\
#             withColumnRenamed('_5', 'rating').\
#             withColumnRenamed('_6', 'reviews').\
#             withColumnRenamed('_7', 'types').\
#             orderBy("similarity", ascending=False)
#         sim_df_filtered = sim_df
#         #sim_df_filtered = filecontents
#         #((sim_df.price_level == price_level_user))
#         print(sim_df_filtered)
#         return redirect(url_for('output', processed_data = sim_df_filtered))       
    return render_template('moreedits.html')
    

@app.route('/output')
def output():
    processed_data = request.args.get('processed_data')
    #print(processed_data)
    # render the output HTML page
    return render_template('onlytable.html', table = processed_data)

app.run(host='localhost', port=6051)


 * Serving Flask app "JSON_OUTPUT" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:6051/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2023 19:43:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2023 19:43:20] "GET /withblurb.png HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2023 19:43:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2023 19:43:29] "POST /output HTTP/1.1" 405 -


In [67]:
import numpy as np

def cossim(v1, v2):
    dot_product = np.sum(v1 * v2)
    mag_v1 = np.sqrt(np.sum(np.power(v1, 2)))
    mag_v2 = np.sqrt(np.sum(np.power(v2, 2)))
    return dot_product / (mag_v1 * mag_v2 + 0.1)


query_txt = "cafe"
query_df = sc.parallelize([(1,query_txt)]).toDF(['index','Types'])
query_tok = regexTokFilter.transform(query_df)
query_vec = model_type.transform(query_tok)
query_vec = query_vec.select('wordvectors_type').collect()[0][0]


sim_rdd = sc.parallelize((i[0], i[1], i[2],i[3],i[4],i[5],i[6], float(cossim(query_vec, i[7]))) for i in sparkDF_wv_final)
sim_df = spark.createDataFrame(sim_rdd).\
        withColumnRenamed('_1', 'Name').\
        withColumnRenamed('_2', 'Address').\
        withColumnRenamed('_3', 'Types').\
        withColumnRenamed('_4', 'Price_level').\
        withColumnRenamed('_5', 'Rating').\
        withColumnRenamed('_6', 'Reviews').\
        withColumnRenamed('_7', 'Neighborhood').\
        withColumnRenamed('_8', 'similarity').\
        orderBy("similarity", ascending=False)
sim_df.show()

+--------------------+--------------------+-----------------+------------------+--------------------+-------+------------+-------------------+
|                Name|             Address|            Types|       Price_level|              Rating|Reviews|Neighborhood|         similarity|
+--------------------+--------------------+-----------------+------------------+--------------------+-------+------------+-------------------+
|Dominique Ansel B...|189 Spring Street...|       40.7252509|       -74.0028734|['bakery', 'cafe'...|      2|           4|0.40787439458754826|
|          McDonald's|160 Broadway, New...|       40.7094789|-74.01003639999999|['cafe', 'restaur...|      1|           4| 0.4078478075310219|
|  Nom Wah Tea Parlor|13 Doyers Street,...|       40.7144447|        -73.998254|['cafe', 'restaur...|      1|           4|0.40781934270171466|
|Buvette Gastrotheque|At the corner of ...|       40.7326259|-74.00432839999999|['cafe', 'bar', '...|      2|           4| 0.4077676173307124|

In [42]:
# THE ONE THAT WORKS
from flask import Flask, request, jsonify, redirect, url_for, render_template
import numpy as np

app = Flask("JSON_OUTPUT")

@app.route('/')
def form():
    return render_template('moreedits.html')

@app.route('/submit', methods=['GET','POST'])
def submit():
    if request.method == ['POST']:
        types_user = request.form['q']
        price_level_user = request.form['price_level']
        neighborhood_user = request.form['neighborhoods']
        rating_user = request.form['rating']
        
    return render_template('onlytable.html',table=filecontents.head().to_html(classes='table'))

@app.route('/output')
def output():
    # render the output HTML page
    return render_template('onlytable.html',table=processed_data)

app.run(host='localhost', port=6076)

 * Serving Flask app "JSON_OUTPUT" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:6076/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2023 20:31:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2023 20:31:13] "GET /withblurb.png HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2023 20:31:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2023 20:31:19] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2023 20:31:19] "GET /headerimage.png HTTP/1.1" 404 -
